In [9]:
import os
import pandas
import numpy
import cv2 as cv
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from tqdm import tqdm

In [ ]:
# dataset site
# https://bop.felk.cvut.cz/datasets/

In [6]:
# %wget http://ptak.felk.cvut.cz/6DB/public/bop_datasets/lm_train_pbr.zip

# export SRC=http://ptak.felk.cvut.cz/6DB/public/bop_datasets
# wget $SRC/lm_base.zip         # Base archive with dataset info, camera parameters, etc.
# wget $SRC/lm_models.zip       # 3D object models.
# wget $SRC/lm_test_all.zip     # All test images ("_bop19" for a subset used in the BOP Challenge 2019/2020).
# wget $SRC/lm_train_pbr.zip    # PBR training images (rendered with BlenderProc4BOP).

# unzip lm_base.zip             # Contains folder "lm".
# unzip lm_models.zip -d lm     # Unpacks to "lm".
# unzip lm_test_all.zip -d lm   # Unpacks to "lm".
# unzip lm_train_pbr.zip -d lm  # Unpacks to "lm".

UsageError: Line magic function `%wget` not found.


#TODO
1. Dataset class (Currently {recyclable: 0, non-recyclable: 1})
2. How to use dataset
3. Crop image from bound box
4. Optimize NN

Process data

In [ ]:
result_size = (100, 100)
camera_parameter = []

def crop(image, bound_box):
    pass

def resize(image):
    return cv2.resize(image, result_size, cv2.INTER_AREA)

In [ ]:

# class Dataset(torch.utils.data.Dataset):
#     def __init__(self):
#         pass

#     def __getitem__(self, index):
#         pass

#     def __len__(self):
#         pass

Train

In [ ]:
class Net(nn.Module):
  def __init__(self):
    super.__init__()

    self.conv1 = nn.Conv2d(1, 32, 5) # input is 1 image, 32 output channels, 5x5 kernel / window
    self.conv2 = nn.Conv2d(32, 64, 5) # input is 32, bc the first layer output 32. Then we say the output will be 64 channels, 5x5 kernel / window
    self.conv3 = nn.Conv2d(64, 128, 5)

    self._to_linear = None

    x = torch.randn(result_size[0], result_size[1]).view(-1,1,result_size[0], result_size[1]) # init
    self._to_linear = None
    self.convs(x)

    self.fc1 = nn.Linear(self._to_linear, 1024)
    self.fc2 = nn.Linear(1024, 2)
    

  def convs(self, x):
    x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
    x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
    x = F.max_pool2d(F.relu(self.conv3(x)), (2, 2))

    if self._to_linear is None:
      self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
      return x

  def forward(self, x):
    x = F.relu(self.fc1(x))
    result = F.softmax(self.fc2(x), dim=1)
    return result

In [ ]:
# ไม่เกี่ยว
def train(net):
    for epoch in range(EPOCHS):
        for i in tqdm(range(0, len(train_X), BATCH_SIZE)): # from 0, to the len of x, stepping BATCH_SIZE at a time. [:50] ..for now just to dev
            #print(f"{i}:{i+BATCH_SIZE}")
            batch_X = train_X[i:i+BATCH_SIZE].view(-1, 1, 50, 50)
            batch_y = train_y[i:i+BATCH_SIZE]

            net.zero_grad()

            outputs = net(batch_X)
            loss = loss_function(outputs, batch_y)
            loss.backward()
            optimizer.step()    # Does the update

        print(f"Epoch: {epoch}. Loss: {loss}")

In [ ]:
lr = 0.001
MAX_EPOCH = 1000
optimizer = optim.Adam(lr=lr)
loss_fn = nn.MSELoss()
loss_history = []
batch_size = 16

def train():
    model = Net()
    for epoch in MAX_EPOCH:
        samples = []
        total_loss = 0
        for X, y in samples:
            optimizer.zero_grad()
            output = model(X)
            loss = loss_fn(X, y)
            loss.backward()
            optimizer.step()
            total_loss += loss
        loss_history.append(total_loss)
        
        

In [ ]:
def test():
    model = Net()
    samples = []
    total_loss = 0
    for X, y in samples:
        output = model(X)
        loss = loss_fn(X, y)
        total_loss += loss
    loss_history.append(total_loss)